In [1]:
from bs4 import BeautifulSoup
import requests
import re
import urllib.request
from urllib.request import urlopen
import os

In [22]:
#soup = BeautifulSoup(open("Best Books Ever (53139 books).html"), features="lxml") 
#not convenient.. let's try to load the page

In [9]:
#Exercise 1.1

web_page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1")

def get_url(url):
    req = urllib.request.Request(url)
    return urllib.request.urlopen(req)

def listToString(s):  
    
    # initialize an empty string 
    str1 = "\n"
    # return string   
    return (str1.join(s)) 

In [44]:
page = 151
books_urls = []
while (page>=151 and page <= 300):
    web_page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page={}".format(page))
    soup = BeautifulSoup(web_page.content, features="lxml")
    for link in soup.find_all('a', class_='bookTitle', itemprop = 'url', href = True):
        url = link.get('href')
        url = "https://www.goodreads.com/"+url
        books_urls.append(url)
    #book_links = print_book_links(soup)
    str_book_links = listToString(books_urls)
    page+=1

In [43]:
str_book_links #it looks orrible now.. but the file will be written correctly

'/book/show/2767052-the-hunger-games\n/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix\n/book/show/2657.To_Kill_a_Mockingbird\n/book/show/1885.Pride_and_Prejudice\n/book/show/41865.Twilight\n/book/show/19063.The_Book_Thief\n/book/show/170448.Animal_Farm\n/book/show/11127.The_Chronicles_of_Narnia\n/book/show/30.J_R_R_Tolkien_4_Book_Boxed_Set\n/book/show/11870085-the-fault-in-our-stars\n/book/show/18405.Gone_with_the_Wind\n/book/show/386162.The_Hitchhiker_s_Guide_to_the_Galaxy\n/book/show/370493.The_Giving_Tree\n/book/show/6185.Wuthering_Heights\n/book/show/968.The_Da_Vinci_Code\n/book/show/5297.The_Picture_of_Dorian_Gray\n/book/show/929.Memoirs_of_a_Geisha\n/book/show/24213.Alice_s_Adventures_in_Wonderland_Through_the_Looking_Glass\n/book/show/10210.Jane_Eyre\n/book/show/24280.Les_Mis_rables\n/book/show/13079982-fahrenheit-451\n/book/show/13335037-divergent\n/book/show/7624.Lord_of_the_Flies\n/book/show/18135.Romeo_and_Juliet\n/book/show/18144590-the-alchemist\n/book/show/22628.Th

In [45]:
#save all links found into a file.txt
f = open("BookLinks.txt", "w")
f.write(str_book_links)
f.close()

In [14]:
#Exercise 1.2

page = 233
htmls = []
while (page>= 233 and page <= 300):
    
    directory = "HTMLSources_Page"+str(page)
    parent_dir = "/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3"
    path = os.path.join(parent_dir, directory)
    os.mkdir(path)
    
    web_page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page={}".format(page))
    soup = BeautifulSoup(web_page.content, features="lxml")
    
    i = 1
    
    for link in soup.find_all('a', class_='bookTitle', itemprop = 'url', href = True):
        url = link.get('href')
        url = "https://www.goodreads.com/"+url
        html = urllib.request.urlopen(url).read()
        html_formatted = html.decode("utf-8")
        
        namefile = "Book"+str(i)+"_"+"Page"+str(page)
        with open(os.path.join(path,namefile), 'w') as f:
            f.write(str(html_formatted))
            f.close()
        i+=1
    
    page+=1

In [81]:
#Exercise 1.3

import codecs

page = 151

i = 1 #book_id

while (page <= 151):
    #let's move to the correct directory --> the directory associated to the current page
    directory = "HTMLSources_Page"+str(page)
    parent_dir = "/home/lorenzo/Documents/Università/1° Term/Algorithmic Methods of Data Mining/LAB/Homeworks/HW3"
    path = os.path.join(parent_dir, directory)
    os.chdir(path)
    
    while (i <= 100):#each page contains 100 books
    
        filename = "Book"+str(i)+"_Page"+str(page)
        #print(filename)
    
        #now I need to read the stored locally html files
        file = codecs.open(filename, 'r', 'utf-8')
        document = BeautifulSoup(file.read(), features="lxml") #"document" contains the html text of the corresp file
        
        print("Title: ",document.find_all("h1", id="bookTitle", class_="gr-h1 gr-h1--serif", itemprop="name")[0].contents[0])
        print("Series: ",document.find_all("a", class_="greyText")[0].contents[0])
        print("Author: ",document.find_all("span", itemprop="name")[0].contents[0])
        print("Rating Value: ",document.find_all("span", itemprop="ratingValue")[0].contents[0])
        print("Rating Count: ",document.find_all("a", class_="gr-hyperlink", href="#other_reviews")[0].contents[2])
        print("Plot: ", ' '.join([c for c in document.find_all('div', id="description")[0].contents[1].contents if isinstance(c, str)]))
        print("Number of Pages: ",document.find_all("span", itemprop="numberOfPages")[0].contents[0])
        print("Publishing Date: ", document.find_all("div", class_="row")[1].contents[0])
        print("Characters: ", document.find_all("a", href = href.split("/")) if href[0] == "characters") ###WRONG! 
        #SETTING
        
        #description = ' '.join([c for c in soup.find_all('div', id="description")[0].contents[1].contents if isinstance(c, str)])
        
        i+=1 #increment book_id
    
    page += 1 #increment page
        
    os.chdir('../') #back to the parent working directory

SyntaxError: invalid syntax (<ipython-input-81-1270d554853b>, line 31)